# Day 2：基于交易量的量化指标 - 使用模块化回测工具

本notebook展示如何使用我们封装的回测工具模块来进行更系统化、专业化的回测。这是对原始notebook `6_使用Backtrader进行回测.ipynb` 的重构版本，使用了模块化的设计来提高代码的可重用性。

## 1. 环境准备和导入模块

首先导入我们封装的回测工具模块以及其他必要的库：

In [ ]:
# 导入基础库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import backtrader as bt
import warnings
import os

# 导入回测工具模块
from utils import get_ts_data, df_to_btfeed, run_backtest
from utils import BaseStrategy, VolumeBreakoutStrategy
from utils import plot_performance_analysis, plot_backtest_results
from utils import optimize_ma_strategy

# 忽略警告
warnings.filterwarnings('ignore')

# 设置显示选项
plt.style.use('seaborn-v0_8')
pd.set_option('display.max_columns', None)

## 2. 获取数据

使用 `get_ts_data` 函数从Tushare获取股票数据或从本地加载已有数据：

In [ ]:
# 设置Tushare Token - 请替换为你自己的token
ts_token = 'your_tushare_token'  # 替换为你的token
ts_code = '000001.SZ'
start_date = '2020-01-01'
end_date = '2022-01-01'

# 优先从本地加载数据
data_dir = './data'
os.makedirs(data_dir, exist_ok=True)
data_file = f'{data_dir}/{ts_code}-{start_date}-{end_date}-30min.csv'

if os.path.exists(data_file):
    df = pd.read_csv(data_file, parse_dates=['trade_time'])
    print(f"从本地文件加载数据: {data_file}")
else:
    # 如果本地文件不存在，则从Tushare获取
    df = get_ts_data(ts_token, ts_code, start_date, end_date, freq='30min')

# 显示数据信息
df.head()

## 3. 数据预处理

对数据进行必要的预处理，以供回测使用：

In [ ]:
# 确保数据按时间排序
df = df.sort_values('trade_time').reset_index(drop=True)

# 查看数据统计信息
print(f"数据时间范围: {df['trade_time'].min()} 至 {df['trade_time'].max()}")
print(f"共 {len(df)} 条记录")
print(f"数据列: {df.columns.tolist()}")

# 显示基本统计信息
df.describe()

## 4. 执行基本回测

使用 `VolumeBreakoutStrategy` 策略进行回测：

In [ ]:
# 定义回测参数
strategy_params = {
    'volume_period': 20,   # 交易量均线周期
    'volume_mult': 2.0,    # 交易量倍数阈值
    'exit_bars': 5,        # 持有周期
    'stop_loss': 0.05,     # 止损比例
    'take_profit': 0.10    # 止盈比例
}

# 执行回测
results, strategy = run_backtest(
    df=df, 
    strategy_class=VolumeBreakoutStrategy, 
    strategy_params=strategy_params,
    initial_cash=100000,
    commission=0.001
)

## 5. 可视化回测结果

使用 `plot_backtest_results` 和 `plot_performance_analysis` 函数可视化回测结果：

In [ ]:
# 绘制回测结果
fig = plot_backtest_results(df, results, max_candles=200)
fig.show()

In [ ]:
# 绘制性能分析图表
fig, table = plot_performance_analysis(results)
fig.show()
table.show()

## 6. 参数优化

使用 `optimize_ma_strategy` 函数进行移动平均策略的参数优化：

In [ ]:
# 将数据转换为Backtrader适用的格式
bt_data = df_to_btfeed(df)

# 定义参数优化范围
ma_short_range = (5, 20)   # 短期均线范围
ma_long_range = (20, 50)   # 长期均线范围
step = 5                   # 步长

# 执行参数优化
opt_results = optimize_ma_strategy(
    data=bt_data,
    ma_short_range=ma_short_range,
    ma_long_range=ma_long_range,
    step=step,
    commission=0.001,
    initial_cash=100000
)

# 显示优化结果
opt_results.head(10)

## 7. 自定义策略回测

基于 `BaseStrategy` 创建自定义策略并进行回测：

In [ ]:
# 定义自定义策略
class CustomStrategy(BaseStrategy):
    params = (
        ('ma_period', 20),      # 移动平均周期
        ('rsi_period', 14),     # RSI周期
        ('rsi_overbought', 70), # RSI超买水平
        ('rsi_oversold', 30),   # RSI超卖水平
        # 继承BaseStrategy的参数
        ('log_level', BaseStrategy.LOG_LEVEL_INFO),
        ('collect_signals', True),
    )
    
    def __init__(self):
        # 调用父类初始化
        BaseStrategy.__init__(self)
        
        # 添加指标
        self.ma = bt.indicators.SMA(self.data.close, period=self.params.ma_period)
        self.rsi = bt.indicators.RSI(self.data.close, period=self.params.rsi_period)
    
    def next(self):
        # 如果没有持仓
        if not self.position:
            # 当价格在MA之上且RSI超卖时买入
            if self.data.close[0] > self.ma[0] and self.rsi[0] < self.params.rsi_oversold:
                # 计算可购买的最大股数
                max_shares = self.calc_max_shares(self.data.close[0])
                if max_shares > 0:
                    self.log(f'买入信号: 价格={self.data.close[0]:.2f}, 数量={max_shares}, RSI={self.rsi[0]:.2f}')
                    self.buy(size=max_shares)
                    self.bar_executed = len(self)
                    self.buy_price = self.data.close[0]
        
        # 如果有持仓
        else:
            current_position_size = self.position.size
            
            # 当价格在MA之下或RSI超买时卖出
            if self.data.close[0] < self.ma[0] or self.rsi[0] > self.params.rsi_overbought:
                self.log(f'卖出信号: 价格={self.data.close[0]:.2f}, 持仓数量={current_position_size}, RSI={self.rsi[0]:.2f}')
                self.close()
                return
            
            # 止损: 亏损超过5%
            if self.data.close[0] < self.buy_price * 0.95:
                self.log(f'止损卖出: 价格={self.data.close[0]:.2f}, 持仓数量={current_position_size}')
                self.close()
                return

In [ ]:
# 执行自定义策略回测
custom_params = {
    'ma_period': 20,
    'rsi_period': 14,
    'rsi_overbought': 70,
    'rsi_oversold': 30
}

custom_results, custom_strategy = run_backtest(
    df=df, 
    strategy_class=CustomStrategy, 
    strategy_params=custom_params,
    initial_cash=100000,
    commission=0.001
)

In [ ]:
# 可视化自定义策略回测结果
fig = plot_backtest_results(df, custom_results, max_candles=200, title='自定义MA+RSI策略回测结果')
fig.show()

## 8. 比较多种策略

比较不同策略的性能：

In [ ]:
# 创建一个表格比较两种策略的性能
comparison = pd.DataFrame([
    {
        '策略': '交易量突破策略',
        '总收益率(%)': results['total_return'],
        '最大回撤(%)': results['max_drawdown'],
        '夏普比率': results['sharpe_ratio'],
        '交易次数': results['total_trades'],
        '胜率(%)': results['winning_trades'] / max(1, results['total_trades']) * 100
    },
    {
        '策略': '自定义MA+RSI策略',
        '总收益率(%)': custom_results['total_return'],
        '最大回撤(%)': custom_results['max_drawdown'],
        '夏普比率': custom_results['sharpe_ratio'],
        '交易次数': custom_results['total_trades'],
        '胜率(%)': custom_results['winning_trades'] / max(1, custom_results['total_trades']) * 100
    }
])

comparison

## 9. 总结和后续优化方向

本notebook展示了如何使用我们封装的回测工具模块进行量化交易策略的回测和评估。通过使用模块化的设计，我们能够：

1. **提高代码重用性**：封装常用函数和类，避免重复编写代码
2. **增强可维护性**：模块化设计使代码更易于维护和更新
3. **简化工作流程**：通过简单的函数调用完成复杂的回测任务

### 后续优化方向：

- **添加更多策略**：开发更多的交易策略类，如网格交易、动量策略等
- **改进评估指标**：加入更多评估指标，如卡玛比率、索提诺比率等
- **多资产回测**：支持同时对多个资产进行回测
- **实时数据接入**：添加实时数据源的支持，为实盘交易做准备
- **机器学习集成**：与机器学习模型集成，实现预测驱动的交易策略